In [1]:
import math
from tensorboardX import SummaryWriter

sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import env_configurations
import games_configurations
env_name = 'SuperMarioBros-v1'#'PongNoFrameskip-v4' #'MountainCar-v0' #'SuperMarioBros-v1'# 'PongNoFrameskip-v4' #SuperMarioBros-v1'
#env_name = 'CartPole-v1'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #

obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
config = games_configurations.mario_config_lstm

observation_shape = obs_space.shape
n_actions = action_space.n
shape =(None, ) + observation_shape
print(shape)
print(n_actions)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


(None, 84, 84, 4)
7


In [4]:
from a2c_discrete import A2CAgent
import tr_helpers
import networks
import ray
ray.init()

2019-07-05 19:10:44,012	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-05_19-10-44_4297/logs.
2019-07-05 19:10:44,129	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:34723 to respond...
2019-07-05 19:10:44,281	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:52771 to respond...
2019-07-05 19:10:44,289	INFO services.py:804 -- Starting Redis shard with 6.74 GB max memory.
2019-07-05 19:10:44,350	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-05_19-10-44_4297/logs.
2019-07-05 19:10:44,352	INFO services.py:1427 -- Starting the Plasma object store with 10.11 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:34723',
 'object_store_address': '/tmp/ray/session_2019-07-05_19-10-44_4297/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-05_19-10-44_4297/sockets/raylet',
 'webui_url': None}

In [5]:


agent = A2CAgent(sess,'skip4', obs_space, True, action_space, config)
#agent.restore('nn/pongSuperMarioBros-v1')
agent.train()


Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
0
1
2
3
4
5
6
7
0
Instructions for updating:
Use tf.cast instead.
(pid=4382) 
(pid=4382) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=4382) For more information, please see:
(pid=4382)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=4382)   * https://github.com/tensorflow/addons
(pid=4382) If you depend on functionality not listed there, please file an issue.
(pid=4382) 
(pid=4382) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=4382)   result = entry_point.lo

Frames per seconds:  399.4028065165552
Frames per seconds:  402.6605325547191
saving next best rewards:  1788.923076923077
Frames per seconds:  399.3980152882727
saving next best rewards:  1824.0666666666666
Frames per seconds:  405.5232494527991
saving next best rewards:  1899.388888888889
Frames per seconds:  387.35774604596725
saving next best rewards:  1998.2173913043478
Frames per seconds:  392.02222723625056
Frames per seconds:  394.428414784263
Frames per seconds:  409.8889536996601
Frames per seconds:  393.7196603003073
Frames per seconds:  402.1722613826853
Frames per seconds:  389.99152418020344
Frames per seconds:  412.59137561221536
Frames per seconds:  397.8638113383064
Frames per seconds:  400.68869497916916
Frames per seconds:  385.1959587030914
Frames per seconds:  396.4142188171463
Frames per seconds:  388.30990205022863
Frames per seconds:  384.8414930207411
Frames per seconds:  379.1991479034498
Frames per seconds:  401.4327656684981
Frames per seconds:  384.53566593

Frames per seconds:  378.5057252180853
Frames per seconds:  324.3031126842218
Frames per seconds:  314.26885843364283
Frames per seconds:  337.01597397259565
Frames per seconds:  348.3333517977098
Frames per seconds:  339.69343647965144
Frames per seconds:  356.3386867786472
Frames per seconds:  380.2588042535835
Frames per seconds:  361.5850541236753
Frames per seconds:  346.7778357943174
Frames per seconds:  346.91769127305145
Frames per seconds:  340.87921283683903
saving next best rewards:  2481.92
Frames per seconds:  345.1283176766829
saving next best rewards:  2514.66
Frames per seconds:  399.39058725462866
saving next best rewards:  2521.81
Frames per seconds:  389.9141997029548
Frames per seconds:  402.3281908311002
Frames per seconds:  356.18490109511725
saving next best rewards:  2534.71
Frames per seconds:  398.94285339291173
Frames per seconds:  390.73339770703706
Frames per seconds:  376.58109772954504
saving next best rewards:  2541.64
Frames per seconds:  374.1413561233

Frames per seconds:  392.99766130297786
Frames per seconds:  391.80490211416617
Frames per seconds:  381.4630204978206
Frames per seconds:  396.0492606040576
Frames per seconds:  371.31648612796
Frames per seconds:  392.66890226035474
Frames per seconds:  403.2569226220594
Frames per seconds:  392.7982562734109
Frames per seconds:  403.28630569184503
Frames per seconds:  375.56965877728277
Frames per seconds:  380.67733569925286
Frames per seconds:  377.1890896237082
Frames per seconds:  380.3017339588924
Frames per seconds:  378.55666798494116
Frames per seconds:  389.6159170607986
Frames per seconds:  326.9523444488588
Frames per seconds:  370.29589548661477
Frames per seconds:  386.2679741659853
Frames per seconds:  399.19466125356445
Frames per seconds:  383.5635040878179
Frames per seconds:  401.2551858540354
Frames per seconds:  391.6673434036835
Frames per seconds:  401.3872962710322
Frames per seconds:  380.0894023099523
saving next best rewards:  2866.32
Frames per seconds:  4

Frames per seconds:  397.93056887920216
Frames per seconds:  256.6310530957858
Frames per seconds:  371.4817552201152
Frames per seconds:  388.74853571024175
Frames per seconds:  396.71069336043485
Frames per seconds:  389.6981441425927
Frames per seconds:  383.60423673786545
Frames per seconds:  337.0970469367456
Frames per seconds:  367.926887616369
Frames per seconds:  386.7896588498238
Frames per seconds:  379.26320439846234
Frames per seconds:  399.70157522706154
Frames per seconds:  395.67607931491045
Frames per seconds:  390.45281471350137
Frames per seconds:  389.64405277535275
Frames per seconds:  247.85650366856387
Frames per seconds:  269.8423465342429
Frames per seconds:  334.78469874954817
Frames per seconds:  366.1018706882546
Frames per seconds:  386.58971681525054
Frames per seconds:  384.7524442716106
Frames per seconds:  406.50524036094913
Frames per seconds:  273.6499585794662
Frames per seconds:  383.40784737841216
Frames per seconds:  362.76835632418175
Frames per 

...

Frames per seconds:  266.3073110563143
Frames per seconds:  136.61297177239615
Frames per seconds:  187.81312648398642
Frames per seconds:  152.5250039392235
Frames per seconds:  297.2749539479868
Frames per seconds:  293.9632020974417
Frames per seconds:  206.70477905088143
Frames per seconds:  276.9951783124087
Frames per seconds:  211.78050457594836
Frames per seconds:  303.48580986352647
Frames per seconds:  216.61595172014802
Frames per seconds:  137.28622090380986
Frames per seconds:  296.68945212956953
Frames per seconds:  130.5781120413728
Frames per seconds:  416.6929632489531
Frames per seconds:  170.33418576426854
Frames per seconds:  200.9036897331118
Frames per seconds:  216.77753257138772
Frames per seconds:  149.1947916969531
Frames per seconds:  265.2818188971777
Frames per seconds:  156.341956789375
Frames per seconds:  262.66368420331634
Frames per seconds:  178.99329884827876
Frames per seconds:  262.3439123379846
Frames per seconds:  149.9518316135991
Frames per sec

Frames per seconds:  218.204694429052
Frames per seconds:  222.6560500826996
Frames per seconds:  216.62394910735878
Frames per seconds:  143.5997359750804
Frames per seconds:  290.47013954372915
Frames per seconds:  150.36166004037565
Frames per seconds:  273.39715296868604
Frames per seconds:  220.50666481977024
Frames per seconds:  142.74409129682482
Frames per seconds:  298.43072852394135
Frames per seconds:  147.1455497433858
Frames per seconds:  397.7589466114523
Frames per seconds:  179.01956784222216
Frames per seconds:  154.8602510262319
Frames per seconds:  220.41947124994854
Frames per seconds:  204.22887255246238
Frames per seconds:  186.58742452003332
Frames per seconds:  262.7692638256535
Frames per seconds:  176.79479245429962
Frames per seconds:  293.07278448082155
Frames per seconds:  143.65757499618024
Frames per seconds:  262.91877914893126
Frames per seconds:  175.5216822420358
Frames per seconds:  220.2446018765188
Frames per seconds:  216.68532576124608
Frames per

Frames per seconds:  205.28726986500456
Frames per seconds:  216.96369429698885
Frames per seconds:  138.71462196311978
Frames per seconds:  187.16555269994575
Frames per seconds:  192.7246840738243
Frames per seconds:  298.6494057353082
Frames per seconds:  202.8280679726186
Frames per seconds:  265.3496551076923
Frames per seconds:  143.5604731120718
Frames per seconds:  275.2449295428496
Frames per seconds:  193.04271004723716
Frames per seconds:  203.68191878838923
Frames per seconds:  272.80022550810145
Frames per seconds:  397.4874617895494
Frames per seconds:  207.34667928747317
Frames per seconds:  146.94699289079063
Frames per seconds:  381.8032429787671
Frames per seconds:  159.03251567657836
Frames per seconds:  395.25845415496184
Frames per seconds:  123.34427844325641
Frames per seconds:  259.4632022497534
Frames per seconds:  233.4133572802588
Frames per seconds:  207.79324438940662
Frames per seconds:  275.8990424078365
Frames per seconds:  179.21719403240172
Frames per 

Exception in thread ray_listen_error_messages:
Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/ray/worker.py", line 1650, in listen_error_messages_raylet
    msg = worker.error_message_pubsub_client.get_message()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=time

KeyboardInterrupt: 

In [6]:
#agent.save('nn/last')
ray.shutdown()

In [ ]:
gym.envs.registry.all()